In [11]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from IPython.display import display

In [12]:
COMMODITY = 'copper_over'
METRIC = 'Recall'
ALGO = 'detection_correction'
# ALGO = 'correction'


EDCR_RESULTS_PATH = f'{COMMODITY}/test/predictions/test_{METRIC}_{ALGO}'
OG_RESULTS_PATH = f'{COMMODITY}/test/results_test.csv'

In [13]:
def extract_info_from_filename(filename):
    parts = filename.split("Rule")
    epsilon = parts[0].split()[-1]
    models_base = parts[1].split("for")
    models = models_base[0].strip()
    base = models_base[1].split("_predictions")[0].strip()
    return base, models, epsilon

In [14]:
def calculate_metrics(df):
    y_true = df['True']
    y_pred = df['Predicted']
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average=None, labels=[0, 1])
    prior = pd.read_csv(OG_RESULTS_PATH, nrows=1)['Prior'].iloc[0]
    return accuracy, precision, recall, f1, prior

In [15]:
def process_files(path):
    all_results = []  
    for filename in os.listdir(path):
        if filename.startswith('Confident') and filename.endswith("_predictions.csv"):
            base, models, confidence = extract_info_from_filename(filename)
            df = pd.read_csv(os.path.join(path, filename))
            accuracy, precision, recall, f1, prior = calculate_metrics(df)
            row = {
                'Base': base,
                'Models': models,
                'Confidence': confidence,
                'Accuracy': accuracy,
                'Precision (0)': precision[0],
                'Recall (0)': recall[0],
                'F1 (0)': f1[0],
                'Precision (1)': precision[1],
                'Recall (1)': recall[1],
                'F1 (1)': f1[1],
                'Prior': prior  
            }
            all_results.append(row)
    
    # Convert the list of results to a DataFrame
    return pd.DataFrame(all_results)

In [16]:
results_df = process_files(EDCR_RESULTS_PATH)

c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to cont

KeyboardInterrupt: 

In [ ]:
# results_df_one = results_df[results_df['Models'] != 'all']
# # pd.set_option('display.max_rows', 200)
# results_df_one

In [ ]:
og_df = pd.read_csv(OG_RESULTS_PATH)
og_df = og_df.drop('Unnamed: 0', axis=1)
og_df = og_df[(og_df['Name'] != 'EDCR')]
pd.set_option('display.max_rows', 400)
og_df

,Name,Params,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),Prior
0,LSTM,256 layers,0.382883,0.828947,0.336898,0.479087,0.150685,0.628571,0.243094,0.16
1,LSTM,128 layers,0.252252,0.800000,0.149733,0.252252,0.149733,0.800000,0.252252,0.16
2,LSTM,64 layers,0.702703,0.845714,0.791444,0.817680,0.170213,0.228571,0.195122,0.16
3,LSTM,32 layers,0.720721,0.857143,0.802139,0.828729,0.212766,0.285714,0.243902,0.16
4,CNN with Attention,"32 filters, kernel size 7",0.720721,0.841530,0.823529,0.832432,0.153846,0.171429,0.162162,0.16
5,CNN with Attention,"32 filters, kernel size 5",0.351351,0.811594,0.299465,0.437500,0.143791,0.628571,0.234043,0.16
6,CNN with Attention,"32 filters, kernel size 3",0.581081,0.826389,0.636364,0.719033,0.128205,0.285714,0.176991,0.16
7,CNN with Attention,"64 filters, kernel size 7",0.729730,0.846995,0.828877,0.837838,0.179487,0.200000,0.189189,0.16
8,CNN with Attention,"64 filters, kernel size 5",0.454955,0.817308,0.454545,0.584192,0.135593,0.457143,0.209150,0.16
9,CNN with Attention,"64 filters, kernel size 3",0.698198,0.848837,0.780749,0.813370,0.180000,0.257143,0.211765,0.16


### Base Model

Generally for lower confidence (0.1-0.3), the rule is basically all '1'. It's always predicting spikes, so that's why lower confidence results look similar to spikes-dumb-model

Conversely, for higher confidence (0.9-0.95), the rule is becoming more like all '0'. So the higher we go, the less spikes we're predicting, hence making it look more like non-spikes-dumb-model

In [ ]:
MODEL_NAME_OF_INTEREST = 'CNN'
MODEL_PARAM_OF_INTEREST = '32_5'

results_df_all = results_df[(results_df['Base'] == f'{MODEL_NAME_OF_INTEREST}_{MODEL_PARAM_OF_INTEREST}') & (results_df['Models'] == 'all') & (results_df['Base'] != "dumb_spikes") & (results_df['Base'] != "dumb_non_spikes")]
results_df_all

,Base,Models,Confidence,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),Prior
19,CNN_32_5,all,0.1,0.157658,0.000000,0.000000,0.000000,0.157658,1.000000,0.272374,0.16
221,CNN_32_5,all,0.2,0.157658,0.000000,0.000000,0.000000,0.157658,1.000000,0.272374,0.16
423,CNN_32_5,all,0.3,0.157658,0.500000,0.010695,0.020942,0.151376,0.942857,0.260870,0.16
625,CNN_32_5,all,0.45,0.202703,0.708333,0.090909,0.161137,0.141414,0.800000,0.240343,0.16
823,CNN_32_5,all,0.4,0.171171,0.600000,0.048128,0.089109,0.140097,0.828571,0.239669,0.16
1025,CNN_32_5,all,0.55,0.288288,0.808511,0.203209,0.324786,0.148571,0.742857,0.247619,0.16
1223,CNN_32_5,all,0.5,0.247748,0.794118,0.144385,0.244344,0.148936,0.800000,0.251121,0.16
1425,CNN_32_5,all,0.6,0.707207,0.850575,0.791444,0.819945,0.187500,0.257143,0.216867,0.16
1627,CNN_32_5,all,0.7,0.842342,0.855140,0.978610,0.912718,0.500000,0.114286,0.186047,0.16
1829,CNN_32_5,all,0.8,0.842342,0.842342,1.000000,0.914425,0.000000,0.000000,0.000000,0.16


### Dumb Model (Only Predicts Non-Spikes)

In [ ]:
results_df_all = results_df[(results_df['Models'] == 'all') & (results_df['Base'] == "dumb_non_spikes")]
results_df_all

,Base,Models,Confidence,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),Prior
24,dumb_non_spikes,all,0.1,0.157658,0.000000,0.000000,0.000000,0.157658,1.000000,0.272374,0.16
226,dumb_non_spikes,all,0.2,0.157658,0.000000,0.000000,0.000000,0.157658,1.000000,0.272374,0.16
428,dumb_non_spikes,all,0.3,0.157658,0.500000,0.010695,0.020942,0.151376,0.942857,0.260870,0.16
630,dumb_non_spikes,all,0.45,0.202703,0.708333,0.090909,0.161137,0.141414,0.800000,0.240343,0.16
828,dumb_non_spikes,all,0.4,0.171171,0.600000,0.048128,0.089109,0.140097,0.828571,0.239669,0.16
1030,dumb_non_spikes,all,0.55,0.288288,0.808511,0.203209,0.324786,0.148571,0.742857,0.247619,0.16
1228,dumb_non_spikes,all,0.5,0.247748,0.794118,0.144385,0.244344,0.148936,0.800000,0.251121,0.16
1430,dumb_non_spikes,all,0.6,0.707207,0.850575,0.791444,0.819945,0.187500,0.257143,0.216867,0.16
1632,dumb_non_spikes,all,0.7,0.842342,0.855140,0.978610,0.912718,0.500000,0.114286,0.186047,0.16
1834,dumb_non_spikes,all,0.8,0.842342,0.842342,1.000000,0.914425,0.000000,0.000000,0.000000,0.16
